In [3]:
import pandas as pd

In [4]:
age_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\age_group_segment.csv")
disorder_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\doctor_disorder_segment.csv")
test_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\test_segment.csv")
dept_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\dept_segment.csv")

In [6]:
import ast

# -----------------------
# 1. Age → doctor list
# -----------------------
user_age_group = 23

def get_age_segment(age: int) -> str:
    if 2 <= age <= 4:
        return "2-4"
    elif 5 <= age <= 12:
        return "5-12"
    elif 13 <= age <= 18:
        return "13-18"
    elif 19 <= age <= 30:
        return "19-30"
    elif 31 <= age <= 45:
        return "31-45"
    elif 46 <= age <= 60:
        return "46-60"
    else:
        return "61+"
    
user_age_segment = get_age_segment(user_age_group)
age_doctors = age_segment.loc[age_segment["age_group"] == user_age_segment, "doctor_id"].values[0]
if isinstance(age_doctors, str):
    age_doctors = ast.literal_eval(age_doctors)
age_doctors = set(age_doctors)

print("Age doctors:", age_doctors)

# -----------------------
# 2. Disorder → doctor list
# -----------------------
user_disorder = "Autism Spectrum Disorder (ASD)"
disorder_doctors = disorder_segment.loc[
    disorder_segment["diagnosed_disorder"] == user_disorder, "doctor_id"
].values[0]

if isinstance(disorder_doctors, str):
    disorder_doctors = ast.literal_eval(disorder_doctors)
disorder_doctors = set(disorder_doctors)

print("Disorder doctors:", disorder_doctors)

# -----------------------
# 3. Test → doctor list
# -----------------------
# For test_segment: directly safe with ast.literal_eval (your data is clean)
test_segment = pd.read_csv(r"C:\Users\aamreen_quantum-i\OneDrive\Desktop\Symptoms_checker\symptoms_checker\CSV Data\Segments\test_segment.csv")
# Convert string representations into Python objects
test_segment["test_combination"] = test_segment["test_combination"].apply(ast.literal_eval)
test_segment["doctors"] = test_segment["doctors"].apply(ast.literal_eval)

def get_doctors_for_test(user_test, test_segment):
    matching_doctors = []
    for _, row in test_segment.iterrows():
        if user_test in row["test_combination"]:
            matching_doctors.extend(row["doctors"])
    return list(set(matching_doctors))  # remove duplicates

user_test = "AQ-10"
test_doctors = set(get_doctors_for_test(user_test, test_segment))

print("Test doctors:", test_doctors)

# -----------------------
# 4. Department → doctor list
# -----------------------
user_department = "Psychology"
dept_segment["doctors"] = dept_segment["doctors"].apply(ast.literal_eval)

dept_doctors = dept_segment.loc[
    dept_segment["department"].str.lower() == user_department.lower(), "doctors"
].values[0]
dept_doctors = set(dept_doctors)

print("Department doctors:", dept_doctors)

# -----------------------
# Final: Intersection
# -----------------------
common_doctors = age_doctors & disorder_doctors & test_doctors & dept_doctors
print("\n✅ Recommended doctors (common across all 4 segments):", common_doctors)


Age doctors: {'D015', 'D016', 'D002', 'D017', 'D006', 'D003', 'D013', 'D011', 'D001', 'D020', 'D007', 'D004', 'D018', 'D005', 'D010', 'D008', 'D012', 'D014', 'D009', 'D019'}
Disorder doctors: {'D015', 'D016', 'D002', 'D017', 'D006', 'D003', 'D013', 'D011', 'D001', 'D020', 'D007', 'D004', 'D018', 'D005', 'D010', 'D008', 'D012', 'D014', 'D009', 'D019'}
Test doctors: {'D015', 'D019', 'D016', 'D002', 'D017', 'D006', 'D003', 'D013', 'D011', 'D001', 'D020', 'D007', 'D004', 'D018', 'D005', 'D010', 'D008', 'D012', 'D014', 'D009'}
Department doctors: {'D015', 'D005', 'D011', 'D016', 'D020', 'D010', 'D007', 'D008', 'D009', 'D006'}

✅ Recommended doctors (common across all 4 segments): {'D015', 'D005', 'D011', 'D016', 'D020', 'D010', 'D007', 'D008', 'D009', 'D006'}


In [2]:
import pandas as pd

# -----------------------------
# 1. Load CSVs
# -----------------------------
# Appointment load table (doctors × daily columns)
appointments_df = pd.read_csv(r"C:\Users\vreddy_quantum-i\Desktop\Symptom_Checker\CSV Data\doctor_appointment_load.csv")

# Leave table (doctor_id + leave dates)
leaves_df = pd.read_csv(r"C:\Users\vreddy_quantum-i\Desktop\Symptom_Checker\CSV Data\leavedata.csv")

# -----------------------------
# 2. Preprocess leave dates
# -----------------------------
# Convert leave dates to datetime for easy comparison
leaves_df["Leave_Dates"] = pd.to_datetime(leaves_df["Leave_Dates"], format="%d-%m-%Y")

# -----------------------------
# 3. Function to check availability
# -----------------------------
def get_doctor_availability(user_date, appointments_df, leaves_df):
    """
    user_date       : str in 'YYYY-MM-DD' format
    appointments_df : wide table with dates as columns
    leaves_df       : long table with doctor_id, leave_date

    Returns: dict {doctor_id: load}
    """
    # Convert input date
    user_date = pd.to_datetime(user_date, format="%Y-%m-%d")

    # Step 1: find doctors on leave that day
    doctors_on_leave = leaves_df.loc[leaves_df["Leave_Dates"] == user_date, "Doctor_ID"].tolist()

    # Step 2: get column name that matches user_date in appointment load
    # Appointment file has columns like "2025-08-25 (Monday)"
    date_str = user_date.strftime("%Y-%m-%d (%A)")

    if date_str not in appointments_df.columns:
        raise ValueError(f"Date {date_str} not found in appointment load table")

    # Step 3: filter appointment load for that date
    result_df = appointments_df[["Doctor ID", "Doctor Name", "Department", date_str]].copy()
    result_df.rename(columns={date_str: "Load"}, inplace=True)

    # Step 4: remove doctors on leave
    result_df = result_df[~result_df["Doctor ID"].isin(doctors_on_leave)]

    # Step 5: convert "Leave" text to 0 load
    result_df["Load"] = pd.to_numeric(result_df["Load"], errors="coerce").fillna(0)

    # Step 6: return dictionary
    return dict(zip(result_df["Doctor ID"], result_df["Load"]))

# -----------------------------
# 4. Example usage
# -----------------------------
user_date = "2025-08-29"  # Friday
availability = get_doctor_availability(user_date, appointments_df, leaves_df)

print(f"Doctor availability on {user_date}:")
print(availability)


Doctor availability on 2025-08-29:
{'D001': 0.0, 'D002': 0.0, 'D003': 0.0, 'D004': 0.0, 'D005': 0.0, 'D006': 0.0, 'D007': 0.0, 'D008': 0.0, 'D009': 50.0, 'D010': 32.0, 'D011': 43.0, 'D012': 0.0, 'D013': 33.0, 'D014': 47.0, 'D015': 0.0, 'D016': 41.0, 'D017': 0.0, 'D018': 0.0, 'D019': 61.0, 'D020': 48.0}
